In [16]:
import df_filter_nvida
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
import re
from unidecode import unidecode
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter

import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

In [2]:
stop_words = set(stopwords.words('english'))

In [6]:
news_articles_data = pd.read_csv("data/us_equities_news_dataset.csv")

In [10]:
# Sample keywords related to NVIDIA and associated companies
nvidia_keywords = [
    'NVDA', 'NVIDIA']

'''

More keywords can be added to the list to improve the filtering process. But adding all those increases the document number significantly making processing slower and creating memory issues that are unsolvable by my mere laptop. 

nvidia_keywords = [
    'NVDA', 'NVIDIA', 'NIO', 'UBER', 'AMZN', 'AMAZON', 'TESLA', 'AI', 'GPU', 'GRAPHICS',
    'CHIP', 'SEMICONDUCTOR', 'AUTONOMOUS', 'DRIVING', 'DEEP LEARNING', 'MACHINE LEARNING'
]

'''

# Compile a regex pattern from the keywords list
nvidia_pattern = '|'.join(nvidia_keywords)  # Combines the keywords into a regex pattern

# Filter articles where the content or ticker column contains any of the keywords
df_news = news_articles_data[
    news_articles_data['content'].str.contains(nvidia_pattern, case=False, na=False)
]

In [11]:
df_news = df_news.drop_duplicates(subset='content', keep='first').reset_index(drop=True)
df_news

,id,ticker,ticker_NVDA_improved,title,title_NVIDIA_topic,category,content,content_NVIDIA_topic,release_date,provider,url,article_id
0,221539,NIO,True,A Central Bank War Just Started And Its Good F...,False,opinion,ECB Effects\nThe move in the euro was huge fa...,True,2019-03-07,Michael Kramer,https://www.investing.com/analysis/a-central-b...,200395687
1,221547,NIO,True,6 Stocks To Watch Nivida Could Be Falling,False,opinion,6 Stocks To Watch March 6 Trading Session\nSt...,True,2019-03-06,Michael Kramer,https://www.investing.com/analysis/6-stocks-to...,200394931
2,221572,NIO,True,Stocks Dow Drops Nearly 400 Points as Apple ...,False,news,Investing com A rout in Apple and Facebook ...,True,2018-11-19,Investing.com,https://www.investing.com/news/stock-market-ne...,1694042
3,221593,UBER,True,The Zacks Analyst Blog Highlights Advanced Mi...,True,opinion,For Immediate ReleaseChicago IL January 13 ...,True,2020-01-12,Zacks Investment Research,https://www.investing.com/analysis/the-zacks-a...,200498277
4,221597,UBER,True,The Best Of CES 2020 Revised,False,opinion,With 4 500 companies bringing their innovation...,True,2020-01-16,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200499164
...,...,...,...,...,...,...,...,...,...,...,...,...
3436,442657,AMD,True,Here s Why Nvidia NVDA Stock Is Gaining Today,True,opinion,Shares of Nvidia NASDAQ NVDA are up nearly...,True,2016-09-27,Zacks Investment Research,https://www.investing.com/analysis/here's-why-...,200155860
3437,442682,AMD,True,4 Stocks To Watch Today ATW CWEI MXL SLCA,False,opinion,It was a pretty good start to the week on Mond...,True,2016-05-17,Harry Boxer,"https://www.investing.com/analysis/atw,-cwei,-...",200130262
3438,442705,AMD,True,Here s What The Buy Side Expects From AMD Thur...,False,opinion,Advanced Micro Devices Inc NYSE AMD is set ...,True,2014-04-17,Estimize,https://www.investing.com/analysis/here’s-what...,209915
3439,442984,T,True,Zacks com Featured Highlights AT T Nu Skin E...,True,opinion,For Immediate Release\n\nChicago IL July 22...,True,2016-07-21,Zacks Investment Research,https://www.investing.com/analysis/zacks.com-f...,200143537


In [12]:
def preprocess_texts(texts, n=0):
    processed_texts = []
    for text in texts:
        # lowercasing, keep text only, remove accents, tokenization
        tokens = [word for word in word_tokenize(re.sub(r'[^a-zA-Z\s]', '', unidecode(text.lower())))]
        # stopword removal
        tokens = [token for token in tokens if token not in stopwords.words('english')]
        processed_texts.append(tokens)
        # remove top-n% and bottom-n% words
    if n > 0:
        word_freq = Counter([word for sentence in processed_texts for word in sentence])
        top_n = set([word for word, _ in word_freq.most_common(int(n/100*len(word_freq)))])
        bottom_n = set([word for word, _ in word_freq.most_common()[:-int(n/100*len(word_freq))-1:-1]])
        processed_texts = [[word for word in sentence if word not in top_n and word not in bottom_n] for sentence in
    processed_texts]
        
    return processed_texts

In [15]:
# Preprocess the filtered news content
processed_content = preprocess_texts(df_news['content'].tolist())

# Show the first few processed texts
processed_content[1:5]

[['ecb',
  'effects',
  'move',
  'euro',
  'huge',
  'falling',
  'pips',
  'huge',
  'important',
  'piece',
  'euro',
  'broke',
  'essential',
  'support',
  'wrote',
  'thoughts',
  'euro',
  'could',
  'even',
  'fall',
  'parity',
  'fed',
  'may',
  'need',
  'cut',
  'rates',
  'later',
  'year',
  'draghi',
  'dovish',
  'today',
  'outlook',
  'much',
  'german',
  'bunds',
  'fell',
  'around',
  'bps',
  'bps',
  'year',
  'fell',
  'break',
  'sends',
  'year',
  'back',
  'december',
  'lows',
  'time',
  'investors',
  'come',
  'realize',
  'rates',
  'remain',
  'low',
  'fed',
  'need',
  'stay',
  'hold',
  'avoid',
  'dollar',
  'strengthen',
  'much',
  'dollar',
  'rise',
  'kill',
  'whatever',
  'inflationary',
  'forces',
  'hurting',
  'multinational',
  'companies',
  'result',
  'may',
  'fed',
  'needs',
  'cut',
  'rates',
  'late',
  'keep',
  'dollar',
  'line',
  'versus',
  'euro',
  'avoid',
  'scenario',
  'good',
  'stocks',
  'investors',
  'move'

In [13]:
def filter_word_from_corpus(data, words):
    # Ensure words is a list, even if a single string is passed
    if isinstance(words, str):
        words = [words]
    # Filter words from data
        filtered_data = [[token for token in row if token not in words] for row in data]
        
    return filtered_data

In [17]:
# Create a dictionary from the processed content
dictionary = corpora.Dictionary(processed_content)

# Convert the list of documents (corpus) into Document Term Matrix using the dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(doc) for doc in processed_content]

# Train the LDA model
num_topics = 3
lda_model = LdaModel(
    doc_term_matrix,
    num_topics=num_topics,
    id2word=dictionary,
    passes=10,  # More passes can improve the model
    random_state=17
)

# Print the topics
for topic in lda_model.print_topics():
    print(topic)

# Get coherence score
coherence_model = CoherenceModel(model=lda_model, texts=processed_content, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f'Coherence Score: {coherence_score:.4f}')

(0, '0.014*"nasdaq" + 0.009*"u" + 0.008*"nyse" + 0.007*"trade" + 0.006*"nvidia" + 0.006*"said" + 0.006*"china" + 0.005*"shares" + 0.005*"year" + 0.005*"index"')
(1, '0.016*"zacks" + 0.016*"year" + 0.014*"company" + 0.011*"quarter" + 0.010*"earnings" + 0.009*"growth" + 0.009*"nvidia" + 0.008*"nasdaq" + 0.007*"rank" + 0.007*"billion"')
(2, '0.009*"nasdaq" + 0.009*"stock" + 0.009*"market" + 0.007*"stocks" + 0.007*"earnings" + 0.007*"week" + 0.005*"p" + 0.005*"investors" + 0.005*"last" + 0.004*"nyse"')
Coherence Score: 0.4414
